In [38]:
import pandas as pd
from konlpy.tag import Okt
from bs4 import BeautifulSoup
import re

okt = Okt()

def preprocess_text(text):
    
    # HTML 태그 제거
    text = BeautifulSoup(text, 'html.parser').get_text()

    # 정규표현식을 사용하여 한글만 남기고 나머지는 제거
    text = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", text)

    # 형태소 분석
    tokens = okt.morphs(text, stem=True)

    # 불용어 제거
    with open('data/한글불용어100.txt') as st:
        lines = st.readlines()
    
    stop_words = []
    for line in lines:
        word = line.split('\t')[0]
        stop_words.append(word)
        stop_words = [line.split('\t')[0] for line in lines]
    
    
    # 형태소 분석기를 돌려서 나온 결과에서 불용어 제거하기
    morphs = okt.morphs(text, stem=True)
    clean_morphs = [morph for morph in morphs if morph not in stop_words]
    clean_morphs
    
    # Feature 변환을 위한 Vectorizer의 입력은 리스트가 아닌 문자열이어야 함
    clean_text = ' '.join(clean_morphs)
    
    # 정제된 텍스트 반환
    return " ".join(tokens)

def process_qa_csv(input_file, output_file):
    # CSV 파일 읽기
    df = pd.read_csv(input_file, encoding='utf-8')

    # '질문'과 '답변' 열에 대해 정제 함수 적용
    df['정제된_질문'] = df['질문'].apply(preprocess_text)
    df['정제된_답변'] = df['답변'].apply(preprocess_text)

    # 질문과 답변을 쌍으로 가지는 형태로 바꾸기
    chatbot_data = [{'질문': q, '답변': a} for q, a in zip(df['정제된_질문'], df['정제된_답변'])]

    # 데이터프레임으로 변환
    chatbot_df = pd.DataFrame(chatbot_data)

    # 정제된 데이터를 CSV 파일로 저장
    chatbot_df.to_csv(output_file, index=False, encoding='utf-8')

# 사용 예시
process_qa_csv('data/팁308 copy.csv', 'data/chatbot_data.csv')

C:\Users\human\AppData\Local\Temp\ipykernel_13664\843051816.py:11: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()
C:\Users\human\AppData\Local\Temp\ipykernel_13664\843051816.py:11: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()


In [39]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt
from konlpy.tag import Komoran

# CSV 파일 불러오기
def load_data(csv_path):
    df = pd.read_csv(csv_path, encoding='utf-8')
    return df

# 텍스트 전처리 함수
def preprocess_text(text, tokenizer):
    tokens = tokenizer.morphs(text)  # 형태소 분석
    return ' '.join(tokens)

# 챗봇 함수 구현
def create_chatbot(csv_path, tokenizer):
    # 데이터 불러오기
    df = load_data(csv_path)

    # 텍스트 전처리
    df['processed_question'] = df['질문'].apply(lambda x: preprocess_text(x, tokenizer))

    # TF-IDF 벡터화
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df['processed_question'])

    # 챗봇 함수
    def chatbot(user_input):
        # 사용자 입력 전처리
        processed_input = preprocess_text(user_input, tokenizer)

        # 사용자 입력을 TF-IDF로 변환
        input_vector = vectorizer.transform([processed_input])

        # 코사인 유사도 계산
        similarities = cosine_similarity(input_vector, tfidf_matrix).flatten()

        # 가장 유사한 질문의 인덱스 찾기
        max_similarity_index = similarities.argmax()

        # 해당 인덱스에 대응하는 답변 출력
        response = df.loc[max_similarity_index, '답변']
        return response

    return chatbot

# 챗봇 생성
csv_path = 'data/chatbot_data.csv'
tokenizer = Okt()  # 형태소 분석기에 맞게 수정
chatbot = create_chatbot(csv_path, tokenizer)

# 테스트
user_input = "월세 주의점."
response = chatbot(user_input)
print(response)


안녕하다 유창 효 공인 중개사 이다 부동산 매매 시 유의 하다 점 은 부동산 을 매매 하다 매도 자 가 실제 권한 이 있다 소유자 인지 그리고 당사자 인지 에 대한 확인 이 중요하다 사 려고 하 는 매물 에 현황 을 정확하다 파악 하다 하다 쉽다 예 로 건축물 대장 토지대장 등기부 등본 을 통해 불법 건축물 여부 면적 토지 점유 하다 권리 근저당 등 을 전부 확인 하다 하다 마지막 으로 분쟁 의 소지 가 예상 되다 부분 은 특약 으로 기재 하 되다 반드시 명확하다 작성 하다 위반 시 계약해지 나 손해배상 등 을 명시 하다 좋다
